# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [4]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [25]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
from IPython.display import clear_output

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta:@carlakiim


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret

    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [6]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter:@carlakiim

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

FileNotFoundError: [Errno 2] No such file or directory: 'auth.pass'

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [ ]:
#Produto escolhido:
produto = 'KKWBeauty'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'en'

Capturando os dados do twitter:

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

In [222]:
#transfomando em listas
#RODAR A PARTIR DAQUI
dftreino = pd.read_excel('./{0}.xlsx'.format(produto))
dfteste = pd.read_excel('./{0}.xlsx'.format(produto), sheetname = 'Teste')

treino = []
teste = []

for i in range (t):
    treino.append([dftreino['Treinamento'][i], dftreino['Relevência'][i]]) 

for i in range (n-t):
    teste.append ([dfteste['Teste'][i], dftreino['Relevência'][i]])


___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [223]:
#funcao do produtorio
def produtorio(n):
    total = 100000
    for i in range(0, len(n)):
        total *= n[i]
    return total

class classificador:
        
    def __init__ (self,treino,teste):
        self.Fsim = 0 #frequencia de tweets relevantes
        self.Fnao = 0 #frequencia de tweets nao relevantes
        self.totalpalavra = [0,0,0] #quantidade total de palavras, palavras relevantes(sim) e palavras irrelevantes(nao)
        self.listaFreq = [] #[palavra, ftotal, fsim, fnao]
        self.treino = treino
        self.contapalavra = 0
        self.teste = teste
        
    def normalize(self):
        for tweet in self.treino:
            tweet[0] = tweet[0].replace(':', '')
            tweet[0] = tweet[0].replace('"','')
            tweet[0] = tweet[0].replace('()','')
            tweet[0] = tweet[0].replace('@','')
            tweet[0] = tweet[0].replace('rt','')
            tweet[0] = tweet[0].replace('  ','')
            tweet[0] = tweet[0].replace('"','')
            
            tweet[0] = tweet[0].split(" ")
        
        for tweet in self.teste:
            tweet[0] = tweet[0].replace(':','')
            tweet[0] = tweet[0].replace('"','')
            tweet[0] = tweet[0].replace('()','')
            tweet[0] = tweet[0].replace('@','')
            tweet[0] = tweet[0].replace('rt','')
            tweet[0] = tweet[0].replace('  ','')
            tweet[0] = tweet[0].replace('"','')
            
            tweet[0] = tweet[0].split(" ")
            
    
    def treinando(self):
        c=0
        for tweet in self.treino:
            c+=1
            clear_output()
            print(str((c/300)*100) +"%")
            if tweet[1] == 'sim':
                self.Fsim += 1
            else:
                self.Fnao += 1
                
            for palavra in tweet[0]:
                onlist = False
                self.totalpalavra[0] += 1
                for FreqPala in self.listaFreq:
                    if palavra == FreqPala[0] and tweet[1] == 'sim':
                        onlist = True
                        FreqPala[1] += 1
                        FreqPala[2] += 1
                        self.totalpalavra[1] += 1
                        
                    elif palavra == FreqPala[0] and tweet[1] == 'não':
                        onlist = True
                        FreqPala[1] += 1
                        FreqPala[3] += 1
                        self.totalpalavra[2] += 1
                        
                if not onlist:
                    self.contapalavra += 1
                    if tweet[1] == 'sim':
                        self.listaFreq.append([palavra, 1, 1, 0])
                        self.totalpalavra[1] += 1
                    else:
                        self.listaFreq.append([palavra, 1, 0, 1])
                        self.totalpalavra[2] += 1
                            
                            
   

    def analise(self,tweet):
        listaph = []
        listaphsim = []
        listaphnao = []
        Psim = self.Fsim/300 #frequencia de um tweet relevante aparecer dentre todos
        Pnao = self.Fnao/300 #frequencia de um tweet  nao relevante aparecer dentre todos
        for palavra in tweet:
            nalista = False
            for FreqPala in self.listaFreq:
                if nalista == False and FreqPala[0] == palavra:
                    nalista = True
                    Ph = (FreqPala[1]+1)/self.contapalavra   # frequencia de cada palavra
                    Phsim = (FreqPala[2]+1)/(self.contapalavra + FreqPala[2]) # frequencia de cada palavra relevante(sim) 
                    Phnao = (FreqPala[3]+1)/(self.contapalavra + FreqPala[3]) # frequencia de cada palavra irrelevante (nao)
                    #print(FreqPala)
                    listaph.append(Ph)
                    listaphsim.append(Phsim)
                    listaphnao.append(Phnao)
                                       
            if not nalista:
                listaph.append(1/self.contapalavra)
                listaphsim.append(1/(self.contapalavra + FreqPala[2]))
                listaphnao.append(1/(self.contapalavra + FreqPala[3]))             
        PH = produtorio(listaph)
        PHSIM = produtorio(listaphsim)
        PHNAO = produtorio(listaphnao)
        
        Bayessim = (PHSIM * Psim)/PH
        Bayesnao = (PHNAO * Pnao)/PH
        if (Bayessim > Bayesnao):
            return "sim"
        else:
            return "não"
        
    def testando (self):
        releErro = 0
        releAcerto = 0
        naoreleErro = 0
        naoreleAcerto = 0
        totalAcerto = 0
        totalErro = 0
        C=0
        for tweet in teste:
            C+=1
            clear_output()
            print(str((C/200)*100) +"%")
            if self.analise(tweet[0]) == tweet[1]:
                totalAcerto += 1
                if tweet[0]=="sim":
                    releAcerto += 1
                else:
                    naoreleAcerto += 1
            else:
                totalErro += 1
                if tweet[0]=="sim":
                    releErro += 1
                else:
                    naoreleErro += 1
                        
        print('Total de erros: ' + str(totalErro))
        print('Total de acertos: ' + str(totalAcerto))
        print()
        print('Positivos falsos:' + str(releErro))
        print('Positivos Verdadeiros:' + str(releAcerto))
        print()
        print('Negativos Falsos:' + str(naoreleErro))
        print('Negativos Verdadeiros:' + str(naoreleAcerto))
        

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [224]:
obj = classificador(treino, teste)




In [225]:
obj.normalize()
obj.treinando()

100.0%


In [226]:
print(obj.treino[1])
obj.testando()

100.0%
Total de erros: 26
Total de acertos: 174

Positivos falsos:0
Positivos Verdadeiros:0

Negativos Falsos:26
Negativos Verdadeiros:174


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


Os resultados obtidos demonstram que há diversos comentários (tweets) que aparentam ser irrelevantes, devido à maneira como são escritos. Mas que na verdade, podem muito relevantes para o feedback de um produto.
O número de negativos falsos obtidos, 26, demosntra que uma má leitura de um comentário pode ocasionar na perda de uma opinião crítica, e estando on-line, pode aparecer mais de uma vez por diversos usuários. Então todos esses comentários seriam marcados como irrelevantes, embora pudecem contribuir com melhoras para o produto. Afinal, toda crítica é válida para uma futura melhoria.
Já o número de negativos verdadeiros mstra que uma boa análise foi realizada, de maneira que a maiora dos comentário realmente foram marcados da maneira correta, quando mostravam um certo padrão de spam, sites fakes, descrição e frequência com que apareciam. Essas menssagens tinham uma recorrência maior e eram comentários vazios, nada era diretamente relacionado à qualidade do produto, nem um review. Assim, eram posts que levariam para outro site, onde supostamente se compraria o produto.

As mensagens de sarcasmo e dupla negação acabam se tornando relevantes. Por mais que sejam comentários que debocham, por detrás há uma crítica, uma opinião do consumidor, que é super valiosa para aqueles que fabricam o produto e querem saber sobre o mercado dele.
O projeto consistiu numa breve análise do protudo no twiter, onde há mensagens a todo momento. Assim, para conseguir informações mais consistentes e relevantes para o produto, um aprofundamento com um banco de dados maior(mais tweets além de outras redes sociais), focado com um certo público alvo, no caso, são majoritariamente mulheres, tornariam os dados mais interessantes. Mas para tal análise, um futuro projeto deveria ser realizado, mirando em aspectos mais específicos sobre o que a empresa deseja saber.  